# Using the pluto MutualFundsIndia data-set

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
mfi <- MutualFundsIndia()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### who are the biggest 5 mutual fund managers?

In [2]:
lastAumDate <- (mfi$AumFundwise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

mfi$AumFundwise() %>%
    filter(PERIOD == lastAumDate) %>%
    select(FUND) %>%
    mutate(AUM = AVG_AUM_WO_FOFD + AVG_AUM_FOFD) %>%
    arrange(desc(AUM)) %>%
    print(n=5)


Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 2]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: desc(AUM)
  FUND                                    AUM
  <chr>                                 <dbl>
1 HDFC Mutual Fund                  33519598.
2 ICICI Prudential Mutual Fund      30816648.
3 SBI Mutual Fund                   26466849.
4 Aditya Birla Sun Life Mutual Fund 24256068.
5 Reliance Mutual Fund              23688506.
# … with more rows


### what are the largest 10 mid-cap funds by AUM?

In [3]:
# Meta to filter for the 'Mid-Cap' category
# AumSchemewise to sort funds by AUM - largest first
# NavTimeSeries to get the start and end dates for which NAVs are available

lastMetaDate <- (mfi$Meta() %>%
    summarize(MAX_TS = max(AS_OF)) %>%
    collect())$MAX_TS[1]

lastSwDate <- (mfi$AumSchemewise() %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

print(lastMetaDate)
print(lastSwDate)

[1] "2019-06-22"
[1] "2019-06-01"


In [4]:
mfi$Meta() %>%
    inner_join(mfi$AumSchemewise(), by="SCHEME_CODE") %>%
    inner_join(mfi$NavTimeSeries(), by="SCHEME_CODE") %>%
    group_by(SCHEME_CODE, SCHEME_NAME, CATEGORY, AVG_AUM_WO_FOFD) %>%
    filter(PERIOD == lastSwDate & AS_OF == lastMetaDate & CATEGORY == 'Mid-Cap') %>%
    summarize(start_dt = min(TIME_STAMP), end_dt = max(TIME_STAMP)) %>%
    arrange(desc(AVG_AUM_WO_FOFD)) %>%
    print(n=10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

ERROR: Error: <SQL> 'SELECT TOP(11) "SCHEME_CODE", "SCHEME_NAME", "CATEGORY", "AVG_AUM_WO_FOFD", MIN("TIME_STAMP") AS "start_dt", MAX("TIME_STAMP") AS "end_dt"
FROM (SELECT "LHS"."SCHEME_CODE" AS "SCHEME_CODE", "LHS"."AS_OF" AS "AS_OF", "LHS"."BENCH_ORIG" AS "BENCH_ORIG", "LHS"."BENCH_NOW" AS "BENCH_NOW", "LHS"."CATEGORY" AS "CATEGORY", "LHS"."EXPENSE" AS "EXPENSE", "LHS"."EXPENSE_RATIO" AS "EXPENSE_RATIO", "LHS"."TURNOVER_RATIO" AS "TURNOVER_RATIO", "LHS"."PERIOD" AS "PERIOD", "LHS"."SCHEME_NAME" AS "SCHEME_NAME.x", "LHS"."AVG_AUM_WO_FOFD" AS "AVG_AUM_WO_FOFD", "LHS"."AVG_AUM_FOFD" AS "AVG_AUM_FOFD", "RHS"."SCHEME_NAME" AS "SCHEME_NAME.y", "RHS"."TIME_STAMP" AS "TIME_STAMP", "RHS"."NAV" AS "NAV"
FROM (SELECT "LHS"."SCHEME_CODE" AS "SCHEME_CODE", "LHS"."AS_OF" AS "AS_OF", "LHS"."BENCH_ORIG" AS "BENCH_ORIG", "LHS"."BENCH_NOW" AS "BENCH_NOW", "LHS"."CATEGORY" AS "CATEGORY", "LHS"."EXPENSE" AS "EXPENSE", "LHS"."EXPENSE_RATIO" AS "EXPENSE_RATIO", "LHS"."TURNOVER_RATIO" AS "TURNOVER_RATIO",

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)